## Environments syntax

#### Motivation

- Bislang haben wir vordefinierte Environmenten wie Frozen Like und Google RecSim verwendet.
- Um RL für unser eigenes Problem zu nutzen, können wir keine dieser Environmenten verwenden.
- Wir müssen unsere eigene Environment mit Python definieren.

#### Gefrorener See Rückblick

- Erinnere dich an die Environment des Gefrorenen Sees aus Modul 1:

In [1]:
import gym
env = gym.make("FrozenLake-v1")

#### Gefrorener See Rückblick

- OpenAI Gym ist quelloffen, also könnten wir uns den [Frozen Lake Quellcode](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py) ansehen.
- Er ist jedoch kompliziert und enthält viel mehr, als wir brauchen.
- Lass uns unsere eigene Environment namens Frozen Pond mit den Grundkomponenten von Frozen Lake erstellen.

#### Komponenten eines Env

Konzeptionelle Entscheidungen:

- Beobachtungsraum
- Handlungsraum

In Python müssen wir zumindest implementieren:

- konstruktor
- `Rücksetzen()`
- `step()`

In der Praxis werden wir vielleicht auch andere Methoden brauchen, wie `render()`

#### Konzeptionelle Entscheidungen

Da wir in diesem Fall den Gefrorenen See nachahmen, sind der Beobachtungsraum und der Aktionsraum bereits festgelegt.

In [2]:
observation_space = gym.spaces.Discrete(16)
action_space = gym.spaces.Discrete(4)

Im weiteren Verlauf des Kurses werden wir uns mit diesen Entscheidungen näher beschäftigen!

#### Verschlüsseln

In [3]:
import gym

class FrozenPond(gym.Env):
    pass

- Beachte, dass wir mit der Unterklasse `gym.Env` beginnen.
- Optional: Du kannst über Objekte, Vererbung und Unterklassen lesen.
- Kurz und bündig: Dies ist eine grundlegende `gym.Env` und wir können ihre Eigenschaften überschreiben.

#### Konstrukteur

- Der Konstruktor wird aufgerufen, wenn wir ein neues `FrozenPond` Objekt erstellen.
- Hier definieren wir den Beobachtungsraum und den Aktionsraum.

In [4]:
import gym

class FrozenPond(gym.Env):
    def __init__(self, env_config=None):
        self.observation_space = gym.spaces.Discrete(16)
        self.action_space = gym.spaces.Discrete(4)        

- Aus Gründen der RLlib-Kompatibilität muss der Konstruktor eine "env_config" aufnehmen 
- Wir werden dieses Argument vorerst ignorieren.

#### Zurücksetzen

- Die nächste Methode, die wir brauchen, ist reset.
- Der Konstruktor setzt permanente Parameter wie den Beobachtungsraum.
- der "Reset" legt jede neue Episode fest.
- Zwischen den beiden gibt es einige Freiheiten, z. B. die Festlegung des Zielorts.
- Wenn sich etwas ändern _könnte_, legen wir es in `reset` fest.

In [5]:
# HIDDEN
import numpy as np

In [6]:
class FrozenPond(gym.Env):
    def reset(self):
        self.player = (0, 0) # the player starts at the top-left
        self.goal = (3, 3)   # goal is at the bottom-right
        
        self.holes = np.array([
            [0,0,0,0], # FFFF 
            [0,1,0,1], # FHFH
            [0,0,0,1], # FFFH
            [1,0,0,0]  # HFFF
        ])
        
        return 0 # to be changed to return self.observation()

#### Zurücksetzen

Testen wir das mal aus:

In [7]:
fp = FrozenPond()

In [8]:
fp.reset()

0

In [9]:
fp.holes

array([[0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0]])

Sieht gut aus.

#### Schritt

- Die letzte Methode, die wir brauchen, ist `step`.
- Dies ist die komplizierteste Methode, die die Kernlogik enthält.
- Erinnere dich daran, dass "step" 4 Dinge zurückgibt:
  1. Beobachtung
  2. Belohnung
  3. Erledigt-Flagge
  4. Extra Info (wird ignoriert)
- Der Übersichtlichkeit halber schreiben wir Hilfsmethoden für Observation, Reward und Done sowie eine zusätzliche Hilfsmethode 

#### Schritt: Beobachtung

Erinnere dich daran, dass die Beobachtung ein Index von 0 bis 15 ist:

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

Wir können dies wie folgt kodieren:

In [10]:
class FrozenPond(gym.Env):
    def observation(self):
        return 4*self.player[0] + self.player[1]

Wenn der Spieler zum Beispiel auf (2,1) steht, geben wir zurück

In [11]:
4*2 + 1

9

Hinweis: Jetzt, wo `self.observation` implementiert ist, sollten wir `reset` in `return self.observation()` statt in `return 0` ändern, um die Codequalität zu verbessern.

#### Schritt: Belohnung

In Anlehnung an das Beispiel des Gefrorenen Sees ist die Belohnung 1, wenn der Agent das Ziel erreicht, und 0, wenn nicht:

In [12]:
class FrozenPond(gym.Env):
    def reward(self):
        return int(self.player == self.goal)

Wir werden diese Belohnungsfunktion später im Modul ändern!

#### Schritt: erledigt

- Wir müssen auch wissen, wann eine Episode abgeschlossen ist 
- Nach Frozen Lake ist die Episode beendet, wenn der Agent das Ziel erreicht oder in den Teich fällt.

In [13]:
class FrozenPond(gym.Env):
    def done(self):
        return self.player == self.goal or self.holes[self.player] == 1

#### Schritt: gültige Standorte

Um die Methode `step` einfacher zu machen, schreiben wir eine Hilfsmethode namens `is_valid_loc`, die überprüft, ob ein bestimmter Ort innerhalb der Grenzen liegt (von 0 bis 3 in jeder Dimension).

In [14]:
class FrozenPond(gym.Env):
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3:
            return True
        else:
            return False

#### Schritt: Zusammenbau

- Mit den obigen Teilen können wir nun die Methode "step" schreiben.
- schritt" nimmt eine _Aktion_ auf, aktualisiert den _Zustand_ und gibt die Beobachtung, die Belohnung, das Erledigt-Flag und zusätzliche Informationen (ignoriert) zurück.
- Erinnere dich daran, wie Aktionen kodiert werden: 0 für links, 1 für unten, 2 für rechts, 3 für oben.
- Wir werden einen **nicht rutschigen** gefrorenen Teich implementieren, d.h. deterministisch und nicht stochastisch.

In [15]:
class FrozenPond(gym.Env):
    def step(self, action):
        # Compute the new player location
        if action == 0:   # left
            new_loc = (self.player[0], self.player[1]-1)
        elif action == 1: # down
            new_loc = (self.player[0]+1, self.player[1])
        elif action == 2: # right
            new_loc = (self.player[0], self.player[1]+1)
        elif action == 3: # up
            new_loc = (self.player[0]-1, self.player[1])
        else:
            raise ValueError("Action must be in {0,1,2,3}")
        
        # Update the player location only if you stayed in bounds
        # (if you try to move out of bounds, the action does nothing)
        if self.is_valid_loc(new_loc):
            self.player = new_loc
        
        # Return observation/reward/done
        return self.observation(), self.reward(), self.done(), {}

#### Erfolg!

- Das war's! Wir haben die notwendigen Teile in Frozen Pond implementiert 
  - konstruktor
  - `Rücksetzen`
  - schritt
- Außerdem fügen wir eine optionale Funktion `render` hinzu, damit wir den Zustand zeichnen können:

In [16]:
class FrozenPond(gym.Env):
    def render(self):
        for i in range(4):
            for j in range(4):
                if (i,j) == self.goal:
                    print("⛳️", end="")
                elif (i,j) == self.player:
                    print("🧑", end="")
                elif self.holes[i,j]:
                    print("🕳", end="")
                else:
                    print("🧊", end="")
            print()

- Zum Spaß werden wir Emojis in unserem Kundenrendering verwenden.
- Spieler ist 🧑, Tor ist ⛳️, gefrorenes See-Segment ist 🧊, Loch ist 🕳.

#### Testen unserer Implementierung

In [17]:
# HIDDEN
from envs_03 import FrozenPond

In [18]:
env = FrozenPond()
env.reset()
env.render()

🧑🧊🧊🧊
🧊🕳🧊🕳
🧊🧊🧊🕳
🕳🧊🧊⛳️


#### Testen unserer Implementierung

Lass uns die Methode "step" testen:

In [19]:
env.step(2) # 0=left / 1=down / 2=right / 3=up

(1, 0, False, {'player': (0, 1), 'goal': (3, 3)})

In [20]:
env.render()

🧊🧑🧊🧊
🧊🕳🧊🕳
🧊🧊🧊🕳
🕳🧊🧊⛳️


Sieht gut aus!

#### Testen unserer Implementierung

Lass uns die beiden Environmenten direkt miteinander vergleichen:

In [21]:
lake = gym.make("FrozenLake-v1", is_slippery=False)
pond = FrozenPond()

lake.reset()
pond.reset()

print("Iter | gym obs / our obs | gym reward / our reward | gym done / our done")
for i, a in enumerate([0, 2, 2, 1, 1, 1, 1, 2]):
    lake_obs, lake_rew, lake_done, _ = lake.step(a)
    pond_obs, pond_rew, pond_done, _ = pond.step(a)
    print("%2d   |      %2d / %2d      |          %d / %d        |      %5s / %5s" % \
          (i, lake_obs, pond_obs, lake_rew, pond_rew, lake_done, pond_done))

Iter | gym obs / our obs | gym reward / our reward | gym done / our done
 0   |       0 /  0      |          0 / 0        |      False / False
 1   |       1 /  1      |          0 / 0        |      False / False
 2   |       2 /  2      |          0 / 0        |      False / False
 3   |       6 /  6      |          0 / 0        |      False / False
 4   |      10 / 10      |          0 / 0        |      False / False
 5   |      14 / 14      |          0 / 0        |      False / False
 6   |      14 / 14      |          0 / 0        |      False / False
 7   |      15 / 15      |          1 / 1        |       True /  True


Sie sehen gleich aus!

#### Testen unserer Implementierung

- RLlib kommt auch mit einem env checker
- Dieser kann uns nicht sagen, ob unsere Environment mit der von Frozen Lake identisch ist
- Aber er führt mehrere nützliche Prüfungen durch:

In [22]:
from ray.rllib.utils.pre_checks.env import check_env

In [23]:
check_env(pond)

2022-08-27 16:08:55,501	WARNING env.py:142 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


- Alle Prüfungen wurden bestanden, bis auf diese Warnung über die maximale Episodenlänge.
- Wir können/sollten diese festlegen, damit die Episoden nicht beliebig lang werden.

#### Maximale Schritte pro Episode

- Um eine maximale Anzahl von Schritten pro Episode festzulegen, können wir einen `gym` _wrapper_ verwenden.
- Wrapper sind bequeme Wege, um Environmenten zu verändern, einschließlich Beobachtungen, Aktionen und Belohnungen.
- Hier verwenden wir den Wrapper `TimeLimit`, um ein Schrittlimit festzulegen.

In [24]:
from gym.wrappers import TimeLimit

pond_5 = TimeLimit(pond, max_episode_steps=5)

Wir können überprüfen, dass es nach 5 Schritten erledigt ist, auch wenn das Ziel nicht erreicht wird:

In [25]:
pond_5.reset()
for i in range(5):
    print(pond_5.step(0))

(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, True, {'player': (0, 0), 'goal': (3, 3), 'TimeLimit.truncated': True})


#### Maximale Schritte pro Episode

Eine vernünftigere Schrittgrenze wäre 50 statt 5.

In [26]:
pond_50 = TimeLimit(pond, max_episode_steps=50)

- Zu deiner Information: Es ist auch möglich, diese Grenze in RLlib zu setzen, nur für Trainingszwecke.
- Dies geschieht mit dem Parameter "horizon" in der Trainer-Konfiguration.

#### Lass uns das Gelernte anwenden!

## Gefrorener Teich Belohnungen
<!-- multiple choice -->

Im Gefrorenen See (und Teich) ist die Belohnung 1, wenn der Agent das Ziel erreicht, und sonst 0. Der Agent muss lernen, die Löcher zu vermeiden, aber es gibt eigentlich keine negative Belohnung, wenn er in ein Loch fällt - es ist die gleiche Null-Belohnung, wie wenn er in ein sicheres Stück gefrorenen See läuft! Warum funktioniert dieses System trotzdem, obwohl die Belohnung für das Laufen in ein Loch oder in trockenes Land dieselbe ist?

- [ ] Sobald der Agent in ein Loch fällt, steckt er fest. Er kann zwar weitere Aktionen durchführen, aber die bringen nichts mehr. Deshalb lernt der Agent, Löcher zu vermeiden.
- [Eine Belohnung von 0 ist die niedrigste mögliche Belohnung; wenn der Agent also eine Belohnung von 0 erhält, wenn er in ein Loch fällt, weiß er sofort, dass es schlecht ist, in ein Loch zu fallen.
- [Die Strafe für das Hineinfallen in ein Loch ist indirekt, da die Episode mit einer Belohnung von Null endet und damit die potenzielle Belohnung von 1 für das erfolgreiche Erreichen des Ziels verwirkt wird. Der Agent lernt, dass er, wenn er in ein Loch fällt, _zukünftige_ Belohnungen einbüßt.
- [RL-Agenten bevorzugen längere Episoden. Wenn der Agent in das Loch fällt, endet die Episode sofort, was der Agent zu vermeiden lernt.

## Teich vs. Labyrinth
<!-- coding exercise -->

Nehmen wir an, wir wollen unsere Teichumgebung in ein _Labyrinth_ umwandeln. In diesem Fall haben wir Wände anstelle von Löchern. Der einzige Unterschied zwischen dem Teich und dem Labyrinth ist das Verhalten von Löchern und Wänden. Wenn du im gefrorenen Teich in ein Loch trittst, endet die Episode. Im Labyrinth bewirkt das Betreten einer Wand nichts (d. h. die Aktion ändert den Standort des Agenten nicht, genau wie der Versuch, über den Rand der Karte zu gehen). Um unseren Gefrorenen See in ein Labyrinth zu verwandeln, müssen wir zwei Methoden ändern: `done` und `is_valid_loc`.

Unten findest du die Methoden `done` und `step`, die wir in den Folien oben gesehen haben. Ändere sie so ab, dass wir jetzt ein Labyrinth mit dem oben beschriebenen Verhalten haben: Wenn du gegen eine Wand läufst, passiert nichts.

Beachte, dass die Klasse `Maze` alle anderen Methoden von `FrozenPond` erbt, also kannst du sie ausprobieren!

In [27]:
# EXERCISE
from envs_03 import FrozenPond


class Maze(FrozenPond):
    def done(self):
        return self.player == self.goal or self.holes[self.player] == 1
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3:
            return True
        else:
            return False
    
pond = FrozenPond()
pond.reset()
pond.step(1)
print(pond.step(2))

maze = Maze()
maze.reset()
maze.step(1)
print(maze.step(2))

(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})
(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})


In [28]:
# SOLUTION
from envs_03 import FrozenPond


class Maze(FrozenPond):   
    def done(self):
        return self.player == self.goal
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3 and not self.holes[location]:
            return True
        else:
            return False
    
pond = FrozenPond()
pond.reset()
pond.step(1)
print(pond.step(2))

maze = Maze()
maze.reset()
maze.step(1)
print(maze.step(2))

(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})
(4, 0, False, {'player': (1, 0), 'goal': (3, 3)})
